<a href="https://colab.research.google.com/github/olegsh60/AI-agent/blob/main/mistral_fixed.ipynb" target="_blank">
  <img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/>
</a>

## Что делает весь код.
	1.	Устанавливает библиотеки
	2.	Загружает большую LLM с 4-битной квантизацией
	3.	Принимает текст-подсказку
	4.	Генерирует продолжение текста
	5.	Показывает результат

 ## 1. Установка библиотек
	•	transformers — основная библиотека Hugging Face для загрузки моделей.
	•	bitsandbytes — библиотека для низкоуровневой квантизации моделей (4-bit, 8-bit).
	•	accelerate — упрощает использование устройств (CPU, GPU) и распределённого запуска.

In [ ]:
# Установка нужных библиотек
!pip install -q bitsandbytes transformers accelerate
!pip uninstall -y bitsandbytes
!pip install bitsandbytes --prefer-binary --no-cache-dir

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 76.1/76.1 MB 11.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 5.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 110.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 92.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 52.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 2.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 5.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 20.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 7.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 MB 6.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.1/21.1 MB 79.8 MB/s eta 0:00:00
Found existing installation: bitsandbytes 0.45.5
Uninstalling bitsandbytes-0.45.5:
  Successfully uninstalled bitsandbytes-

## 2. Импорт библиотек
	•	torch — используется для работы с тензорами и устройства (cuda, cpu).
	•	AutoModelForCausalLM — загрузка модели для генерации текста (CausalLM = causal language modeling).
	•	AutoTokenizer — преобразует текст в токены и обратно.
	•	BitsAndBytesConfig — конфигурация для квантизации.

In [ ]:
# Импорт
import torch
from transformers import AutoModelForCausalLM, AutoTokenizer, BitsAndBytesConfig

## Аппаратный ускоритель: GPU (проверка)

In [ ]:
!nvidia-smi

Wed May 14 06:44:08 2025       
+-----------------------------------------------------------------------------------------+
| NVIDIA-SMI 550.54.15              Driver Version: 550.54.15      CUDA Version: 12.4     |
|-----------------------------------------+------------------------+----------------------+
| GPU  Name                 Persistence-M | Bus-Id          Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |           Memory-Usage | GPU-Util  Compute M. |
|                                         |                        |               MIG M. |
|=========================================+========================+======================|
|   0  Tesla T4                       Off |   00000000:00:04.0 Off |                    0 |
| N/A   44C    P8              9W /   70W |       2MiB /  15360MiB |      0%      Default |
|                                         |                        |                  N/A |
+-----------------------------------------+-----

## 3. Настройка 4-битной квантизации
Эта конфигурация позволяет сильно уменьшить объём памяти, занимаемый моделью, и ускорить работу на GPU.

In [ ]:
# Конфигурация квантизации (4-bit)
quant_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_use_double_quant=True,
    bnb_4bit_quant_type="nf4",
    bnb_4bit_compute_dtype=torch.float16
)

Загружается 7-миллиардная языковая модель Mistral, обученная OpenOrca. Она автоматически размещается на доступном устройстве и работает с 4-битной квантизацией.

In [ ]:
# Загрузка модели с автоопределением устройства
model_name = "Open-Orca/Mistral-7B-OpenOrca"

model = AutoModelForCausalLM.from_pretrained(
    model_name,
    device_map="auto",
    torch_dtype=torch.float16,
    quantization_config=quant_config
)

ImportError: Using `bitsandbytes` 4-bit quantization requires the latest version of bitsandbytes: `pip install -U bitsandbytes`

## ❗ Если не работает:

Возможно, тебе выдана Colab-среда без GPU (в бесплатном доступе это бывает часто). Тогда:  
	•	Либо попробуй перезапустить среду несколько раз.  
	•	Либо оформи подписку на Colab Pro, чтобы получать T4/A100 стабильно.  
	•	Либо не используй bitsandbytes: загрузи модель в float16, но без 4-bit квантования



## Пример fallback-кода (без quantization):

In [ ]:
model = AutoModelForCausalLM.from_pretrained(
    "Open-Orca/Mistral-7B-OpenOrca",
    device_map="auto",
    torch_dtype=torch.float16  # Без BitsAndBytesConfig
)

pytorch_model.bin.index.json:   0%|          | 0.00/23.9k [00:00<?, ?B/s]

Fetching 2 files:   0%|          | 0/2 [00:00<?, ?it/s]

pytorch_model-00001-of-00002.bin:   0%|          | 0.00/9.94G [00:00<?, ?B/s]

pytorch_model-00002-of-00002.bin:   0%|          | 0.00/4.54G [00:00<?, ?B/s]

model.safetensors.index.json:   0%|          | 0.00/25.1k [00:00<?, ?B/s]

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

generation_config.json:   0%|          | 0.00/120 [00:00<?, ?B/s]

## 5. Загрузка токенизатора
Токенизатор преобразует текст в числовой формат (токены), который понимает модель.

In [ ]:
# Токенизатор
tokenizer = AutoTokenizer.from_pretrained(model_name, use_fast=True)

tokenizer_config.json:   0%|          | 0.00/1.69k [00:00<?, ?B/s]

tokenizer.model:   0%|          | 0.00/493k [00:00<?, ?B/s]

added_tokens.json:   0%|          | 0.00/90.0 [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/101 [00:00<?, ?B/s]

## 6. Подготовка входного текста
	•	Текст преобразуется в токены
	•	.to("cuda") — отправляется на GPU (если доступен)

## 7. Генерация текста

	•	Результат переводится обратно в читаемый текст
	•	Убираются специальные токены (например, <pad>, <eos>)

In [ ]:
# Генерация текста
prompt = "Расскажи интересный факт о квантовой физике."
inputs = tokenizer(prompt, return_tensors="pt").to("cuda")

with torch.no_grad():
    outputs = model.generate(**inputs, max_new_tokens=100)

print(tokenizer.decode(outputs[0], skip_special_tokens=True))

Setting `pad_token_id` to `eos_token_id`:32000 for open-end generation.


Расскажи интересный факт о квантовой физике.

Квантовая физика - это наука, которая изучает микроскопические объекты, такие как атомы и элементарные частицы. Одним из самых интересных и парадоксальных аспектов квантовой физики является принцип неопределенности. Этот принцип утверждает, что
